In [1]:
from collections import deque
import warnings

import pandas as pd
from loguru import logger

warnings.simplefilter(action='ignore', category=FutureWarning)

from edgedroid.models.timings import *

logger.enable("edgedroid")

class PowerFitTestModel(MultiCurveFittingExecutionTimeModel):
    _fit_functions = (PowerFit,)

class SquareFitTestModel(MultiCurveFittingExecutionTimeModel):
    _fit_functions = (SquareFit,)

class CubeFitTestModel(MultiCurveFittingExecutionTimeModel):
    _fit_functions = (CubeFit,)

class ExponentialFitTestModel(MultiCurveFittingExecutionTimeModel):
    _fit_functions = (ExponentialFit,)
    
models = {
    "multi_curve": MultiCurveFittingExecutionTimeModel,
    "a * x^b + c": PowerFitTestModel,
    "square": SquareFitTestModel,
    "cube": CubeFitTestModel,
    "exp": ExponentialFitTestModel
}

fit_mse_rows = deque()
for name, model_cls in models.items():
    for neuro in (0, 1):
        model = model_cls(neuroticism=neuro)

        for duration, fn in model._exec_time_funcs.items():
            fit_mse_rows.append({"model": name, "neuro": neuro, "duration": duration, "fn": fn.name, "mse": fn.mse})
        
fit_mses = pd.DataFrame(fit_mse_rows)
fit_mses

2024-06-25 09:37:02.021 | DEBUG    | edgedroid.models.timings.curve:__init__:121 - Curve fitting aggregation function: outliers_to_nan_then_mean
2024-06-25 09:37:02.104 | INFO     | edgedroid.models.timings.curve:__init__:146 - Fitting execution time functions to data...
2024-06-25 09:37:02.138 | INFO     | edgedroid.models.timings.curve:__init__:154 - New best fit function for duration [-inf, 5.0): a * x^b + c (MSE: 0.04, prev. MSE 0.04)
2024-06-25 09:37:02.142 | INFO     | edgedroid.models.timings.curve:__init__:154 - New best fit function for duration [-inf, 5.0): a * x^3 + b * x^2 + c * x + d (MSE: 0.03, prev. MSE 0.03)
2024-06-25 09:37:02.157 | INFO     | edgedroid.models.timings.curve:__init__:154 - New best fit function for duration [5.0, 9.0): a * x^b + c (MSE: 0.09, prev. MSE 0.09)
2024-06-25 09:37:02.162 | INFO     | edgedroid.models.timings.curve:__init__:154 - New best fit function for duration [5.0, 9.0): a * x^3 + b * x^2 + c * x + d (MSE: 0.07, prev. MSE 0.07)
2024-06-25

,model,neuro,duration,fn,mse
0,multi_curve,0,"[-inf, 5.0)",a * x^3 + b * x^2 + c * x + d,0.030029
1,multi_curve,0,"[5.0, 9.0)",a * x^3 + b * x^2 + c * x + d,0.069618
2,multi_curve,0,"[9.0, inf)",a * x^3 + b * x^2 + c * x + d,0.041671
3,multi_curve,1,"[-inf, 5.0)",a * x^3 + b * x^2 + c * x + d,0.028126
4,multi_curve,1,"[5.0, 9.0)",a * x^3 + b * x^2 + c * x + d,0.053862
5,multi_curve,1,"[9.0, inf)",a * x^3 + b * x^2 + c * x + d,0.092311
6,a * x^b + c,0,"[-inf, 5.0)",a * x^b + c,0.043871
7,a * x^b + c,0,"[5.0, 9.0)",a * x^b + c,0.089026
8,a * x^b + c,0,"[9.0, inf)",a * x^b + c,0.049558
9,a * x^b + c,1,"[-inf, 5.0)",a * x^b + c,0.035785


In [5]:
agg_fit_mses = fit_mses.groupby(["model", "neuro"])["mse"].mean()
agg_fit_mses

model        neuro
a * x^b + c  0        0.060819
             1        0.099469
cube         0        0.047106
             1        0.058099
exp          0        0.097567
             1        0.116410
multi_curve  0        0.047106
             1        0.058099
square       0        0.061581
             1        0.109304
Name: mse, dtype: float64

In [6]:
print(agg_fit_mses.to_latex(float_format="\\(%0.03f\\)", caption="Average fit MSEs for the realistic models"))

\begin{table}
\caption{Average fit MSEs for the realistic models}
\begin{tabular}{llr}
\toprule
 &  & mse \\
model & neuro &  \\
\midrule
\multirow[t]{2}{*}{a * x^b + c} & 0 & \(0.061\) \\
 & 1 & \(0.099\) \\
\cline{1-3}
\multirow[t]{2}{*}{cube} & 0 & \(0.047\) \\
 & 1 & \(0.058\) \\
\cline{1-3}
\multirow[t]{2}{*}{exp} & 0 & \(0.098\) \\
 & 1 & \(0.116\) \\
\cline{1-3}
\multirow[t]{2}{*}{multi_curve} & 0 & \(0.047\) \\
 & 1 & \(0.058\) \\
\cline{1-3}
\multirow[t]{2}{*}{square} & 0 & \(0.062\) \\
 & 1 & \(0.109\) \\
\cline{1-3}
\bottomrule
\end{tabular}
\end{table}

